<a href="https://colab.research.google.com/github/Grottersha123/intellectual-psy-report/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%81%D0%B2%D0%BE%D0%B1%D0%BE%D0%B4%D0%BD%D1%8B%D1%85_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
cd /content/drive/My Drive/data_psyho


/content/drive/My Drive/data_psyho


In [4]:
pip install nltk

In [5]:
import pandas as pd

import numpy as np
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re

Using TensorFlow backend.


In [6]:
#Загружаем данные 
psycho = pd.read_csv('psychologists.csv', sep=',', error_bad_lines=False, encoding='cp1251')
psycho = psycho.rename(columns=psycho.iloc[0])
psycho = psycho.iloc[1:]
completed = pd.read_csv('completed-psy.csv', sep=',', error_bad_lines=False, encoding='cp1251')
print(psycho.shape, completed.shape)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,15,40,41,46,47,52,53,58,59,64,65,70,71,76,77,82,83,88,89,94,95,100,101,106,107,112,113,118,119,124,125,130,131,141,142,147,148,153,154,159,160,165,166,171,172,177,178,183,184,189,190,200,201,206,207,212,213,218,219,224,225,230,231,236,237,242,243,248,249,254,255,260,261,266,267,272,273,278,279,284,285,290,291,296,297,302,303,308,309,314,315,325,326,331,332,337,338,343,344,349,350,355,356,361,362,367,368,373,374,379,380,385,386,391,392,397,398,408,409,414,415,420,421,426,427,432,433,438,439,444,445,455,456,461,462,467,468,473,474,479,480,485,486,496,497,502,503,508,509,514,515,525,526,531,532,537,538,543,544,549,550,555,556,561,562,567,568,573,574,579,580,585,586,591,592,597,598,603,604,609,610,615,616,621,622,627,628,633,634,639,640,645,646,651,652,657,658,663,664,669,670,675,676,681,682,687,688,693,694,699,700,710,711,716,717,722,723,728,729,734,735,740,741,746,747,752,753,758,75

(16854, 1728) (86, 4)


In [7]:
psycho.head(10)
%clear

In [0]:
# Фильтр по Свердловской области, второй датасет фильтр по россии 
def prepare_data(region):
  psyho_reg = psycho[(psycho.region == str(region))  | (psycho.region == int(region)) ]
  psyho_russia = psycho[(psycho.region != str(region))  | (psycho.region != int(region)) ]
  clmt = list(psyho_reg.columns)
  columns = []
  for i in clmt:
    try:
      if i != None and 'PQ' in i and not 'Time' in i and not 'order' in i :
        columns.append(i)
    except Exception as e:
      print(e, psyho_reg[i])
  psyho_reg_columns = psyho_reg[columns]
  psyho_russia_columns = psyho_russia[columns]
  print(psyho_russia_columns.shape, psyho_reg_columns.shape)
  return psyho_russia_columns, psyho_reg_columns


In [9]:
data_rus, data_reg = prepare_data(66)

argument of type 'numpy.float64' is not iterable 3        1.0
4        1.0
5        1.0
7        1.0
8        1.0
        ... 
14398    1.0
15331    1.0
15505    1.0
16017    1.0
16499    1.0
Name: 501.0, Length: 575, dtype: float64
(16854, 285) (575, 285)


In [0]:
# Первое что нужно сделать это выбрать данные нужные нам. 

In [0]:
data_rus_10 = data_rus.iloc[:1000000]
data = data_rus_10.dropna()
#Делим вопросы со свободным ответом и с вариантом
def select_cloumns(data,data_all):
  clmns = data.columns
  dig = []
  for col in data.columns:
    try:
      for d in data[col]:
        if d.isdigit() and 'Other' not in col or ('PQ601' in col) or ('PQ603' in col):
          dig.append(col)
          break
    except:
      print(d,col)
  not_dig = [i for i in clmns if i not in dig]
  return data_all[dig], data_all[not_dig]


In [0]:
data_rus_num, data_rus_text = select_cloumns(data, data_rus)
data_reg_num, data_reg_text = select_cloumns(data, data_reg)


In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('russian')
stop_words.remove('не')
stemmer = SnowballStemmer("russian")

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub(r'[^a-яА-Я,]', ' ', text)
    text = nltk.word_tokenize(text)    
    tokens = [token.strip() for token in text]  
    text = ' '.join([token for token in tokens if token not in stop_words])
    # text = ' '.join([stemmer.stem(word) for word in text.split(' ') if word not in stop_words])
    text = text.split(',')
    return text
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
%clear

In [15]:
data_rus_text.head(10)

,PQ101-Other,PQ102-Other,PQ103-Other,PQ104-Other,PQ105-Other,PQ106-Other,PQ107-Other,PQ108-Other,PQ109-Other,PQ201-02,PQ301-Other,PQ302-Other,PQ303-Other,PQ304-Other,PQ401-Other,PQ402-Other,PQ403-Other,PQ404-Other,PQ501-Other,PQ503-Other,PQ504-Other,PQ505-Other,PQ506-Other,PQ606-Other,PQ607-Other,PQ614
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"приказы по учреждению, локальные акты",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,нормативно-правовая и материально-техническая ...
2,Экспертная работа,Работа с обучающимися с девиантными поведениями,Ребёнок с аутоагрессивным поаедением,Центром психического здоровья детей и подростков,У молодых психологов отсутствует мотивация,Работаю в ПМСС центре,Организатор,Привлечения педагога психолога не по компетенции,Ведение в расписание уроков психологии,Положение о социальной психологической службе,Старший педагога психолог центра,Старший педагог -психолог,Журнал учёта деятельности педагога-психолога,Отдел дополнительного образования и воспитания...,Литература,Работаю в ПМСС центре,Работаю в ПМСС центре,Работаю в ПМСС центре,Условия нет,"Не хватает ноутбуков, техники",Условий нет,Нет,Сенсорная комната,NaN,NaN,Наличие ещё ПМСС центров
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Адаптированная основная общеобразовательная пр...,NaN,NaN,NaN,NaN,NaN,NaN,Во второй половине дня. Если необходимо прихож...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,уроки развития психомоторики и сенсорных проц...,NaN,NaN,NaN,Положение о карте индивидуального развития обу...,NaN,NaN,Журналы учета видов работы: диагностика; конс...,"в ТКДН и ЗП по детям, находящимся в СОП",NaN,NaN,NaN,NaN,NaN,NaN,Оборудование для сенсорной комнаты: такти...,NaN,NaN,NaN,NaN,увеличение количества ставок педагогов-психоло...
5,Подготовка наглядного материала для индивидуал...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"План работы на год, утвержденный руководителем...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Материально-техническое оснащение
6,NaN,NaN,NaN,NaN,NaN,социальная работа,NaN,NaN,NaN,организационно - методическая,NaN,NaN,NaN,NaN,NaN,NaN,проведение занятий во время доп.образования. з...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,продолжать совершенствовать в ногу со временем
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Устав,трудовой договор",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Организовать отдел медиативной службы ,штат пс..."
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Положение о деятельности педагога-психолога,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Нет пожеланий
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,приказы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"отчетов меньше, зарплаты побольше и так работа..."
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Устав школы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,нет


In [16]:
from collections import Counter
temp = []
for i in data_rus_text.columns:
  data = data_rus_text[i].dropna().apply(lambda x: preprocess_text(x))
  print(data.shape[0], i)
  temp.append((data.shape[0], i))
  # print(data.shape)
  # words = ' '.join(list(data)).split(' ')
  # Counter(words)

1533 PQ101-Other
1126 PQ102-Other
591 PQ103-Other
279 PQ104-Other
292 PQ105-Other
3694 PQ106-Other
1459 PQ107-Other
269 PQ108-Other
645 PQ109-Other
16826 PQ201-02
1043 PQ301-Other
1016 PQ302-Other
4202 PQ303-Other
1208 PQ304-Other
827 PQ401-Other
394 PQ402-Other
1192 PQ403-Other
346 PQ404-Other
608 PQ501-Other
1555 PQ503-Other
1151 PQ504-Other
308 PQ505-Other
1872 PQ506-Other
270 PQ606-Other
1613 PQ607-Other
16796 PQ614


In [17]:
import pandas as pd
import yaml

# пояснение + количество отвечавших вытаскиеваем Other
count_text_name = []
with open(r'dictionary-psy.yml') as f:
    data = yaml.load(f, Loader=yaml.Loader)
    count_answer = []
    text_answer = []
    new_dict = dict()
    print(data)
    for index,d in enumerate(data[:]):
      try:
        a = d['Ответы']
        if len(a) == 4:
          pass
        if len(a) not in count_answer:
          count_answer.append(len(a))
      except KeyError:
        pass
        for count, key in temp:
          if key == d['Ключ']:
            if 'PQ201-02' == d['Ключ']:
              break
            if 'Other' in d['Ключ']:
              if 'PQ304-Other' == d['Ключ']:
                count_text_name.append(('Отчитывание', count,key))
                print('Отчитывание', key, count)
                print(key, data[index-1]) 
                break
              else:
                 count_text_name.append((data[index-1]["Вопрос"], count, key))
                 break
              
            else:
              count_text_name.append((data[index]["Вопрос"], count, key))
              print(d)
              break
        text_answer.append(d)
    print(count_answer)

[{'Вопрос': 'I.1 Оцените затраты своего времени на работу в рамках каждого из нижеперечисленных видов деятельности', 'Ответы': {'Почти не уделяю этому время': 1, 'Немного времени уделяю': 2, 'Много времени уделяю': 3, 'Почти все время уделяю': 4}, 'Подвопросы': [{'Подвопрос': 'Социально-посредническая деятельность (налаживание эффективной коммуникации между людьми и/или группами в ситуации конфликта)', 'Ключ': 'PQ101-01'}, {'Подвопрос': 'Психодиагностика обучающихся индивидуальная, в т.ч. профориентация', 'Ключ': 'PQ101-02'}, {'Подвопрос': 'Психодиагностика обучающихся групповая: в т.ч. профориентация', 'Ключ': 'PQ101-03'}, {'Подвопрос': 'Индивидуальная коррекционно-развивающая работа с обучающимися', 'Ключ': 'PQ101-04'}, {'Подвопрос': 'Групповая коррекционно-развивающая работа с обучающимися', 'Ключ': 'PQ101-05'}, {'Подвопрос': 'Индивидуальная работа с родителями', 'Ключ': 'PQ101-06'}, {'Подвопрос': 'Групповая работа с родителями', 'Ключ': 'PQ101-07'}, {'Подвопрос': 'Индивидуальная ра

In [18]:
ls

 all_text.txt                   directors_is_cities_reg.csv
 completed-psy.csv              psychologists.csv
 dictionary-directors.yml       psyho_is_cities.csv
 dictionary-psychologists.yml   psyho_is_cities_reg.csv
 dictionary-psy.yml             template.xls
 directors_.csv                 word_cloud/
 directors_is_cities            word_cloud16854/
 directors_is_cities.csv       'сравнение ЛО и РФ.xls'


In [19]:
%clear

In [0]:
#  Пояснение: "Оцените затраты своего времени на работу в рамках описанного Вами вида деятельности"
#     Ключ: "PQ101-17"
#  Пояснение: "Введите иную причину, если её нет в списке выше"
  #   Ключ: "PQ105-Other"
  # -
  #   Пояснение: "Оцените, насколько значима указанная Вами причина"
  #   Ключ: "PQ105-07"

In [21]:
import plotly.graph_objects as go
count_text_name = sorted(count_text_name, key=lambda tup: tup[1])
x = [i[2] for i in count_text_name ]
y = [i[1] for i in count_text_name]
# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=x, y=y,
            hovertext=[i[0] for i in count_text_name])])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.9, textposition='auto')
fig.update_layout(title_text='Количество отвеченных вопросов')
fig.show()

In [22]:
!pip install dostoevsky
%clear

     |████████████████████████████████| 225kB 13.8MB/s 
     |████████████████████████████████| 3.0MB 57.6MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 24.2MB 101kB/s 
     |████████████████████████████████| 109.2MB 107kB/s 
     |████████████████████████████████| 757kB 21.7MB/s 
     |████████████████████████████████| 7.1MB 55.1MB/s 
     |████████████████████████████████| 491kB 63.9MB/s 
     |████████████████████████████████| 3.2MB 28.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2384550 sha256=c06eab8c5ac27f44995c107dde0a95619231b3738dbaae26af4974daf1c38769
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-cp36-none-any.whl size=24634 sha256=a891b5c6a86c2563b68cb55be7880e5b77359d5a9f8d84379e3449051591a4b7
  

In [0]:
!dostoevsky download fasttext-social-network-model

In [0]:
data_rus_text['PQ614']

def return_batch(data):
  keys = data.keys()
  neutral = data['neutral'] if 'neutral' in keys else 0
  speech =  data['speech'] if 'speech' in keys else 0
  skip =  data['skip'] if 'skip' in keys else 0
  negative =  data['negative'] if 'negative' in keys else 0
  positive =  data['positive'] if 'positive' in keys else 0
  unknown =  data['unknown'] if 'unknown' in keys else 0
  return neutral, speech, skip,unknown, negative, positive


In [0]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

import plotly.graph_objects as go
def bar_tone_analysise(x,y,key='',question=''):
  # Use the hovertext kw argument for hover text
  fig = go.Figure(data=[go.Bar(x=x, y=y, text=y)])
  # Customize aspect
  fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, opacity=0.9, textposition='auto')
  fig.update_layout(title=key + ' \n' + question)
  fig.show()

def listmerge2(lstlst):
    all=[]
    for lst in lstlst:
      all=all+lst
    return all

def analyze_sentiment(data, text='гит далее это заметно облегчит работу психолога школе увеличит продуктивность организовать бесплатные курсы повышения квалификации столичных вузах организовать большие форумы психологов международном уровне обмена инновационными идеями'):
  def average(data):
    return round(sum(data)/len(data) * 100, 2)
  tokenizer = RegexTokenizer()
  tokens = tokenizer.split(text)  # [('всё', None), ('очень', None), ('плохо', None)]
  result = data.dropna().apply(lambda x: preprocess_text(x))
  result = listmerge2(result.to_list())
  result = [i for i in result if i != '']
  model = FastTextSocialNetworkModel(tokenizer=tokenizer)
  messages = [i for i in  result if i != '']
  results = model.predict(messages, k=3)

  neutral = []
  speech = []
  skip = []
  negative = []
  positive = []
  unknown = []
  all_metric = []
  for message, sentiment in zip(messages, results):
      """
      привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
      я люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
      малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]
      """
      keys = sentiment.keys()
      result = return_batch(sentiment)
      all_metric.append(result)
      if 'neutral' in keys:
        neutral.append(sentiment['neutral'])
      elif 'speech'in keys:
        speech.append(sentiment['speech'])
      elif 'positive' in keys:
        positive.append(sentiment['positive'])
      elif 'negative' in keys:
        negative.append(sentiment['negative'])
      elif 'unknown' in keys:
        unknown.append(sentiment['unknown'])
      elif 'skip' in keys:
        unknown.append(sentiment['skip'])
      
      # print(message, '->', sentiment)
  # data_rus_text['PQ614'].dropna().apply(lambda x: preprocess_text(x))


  neutral = [i[0] for i in all_metric]
  positive = [i[-1] for i in all_metric]
  negative = [i[-2] for i in all_metric]
  skip = [i[2] for i in all_metric]
  speech = [i[1] for i in all_metric]
  unknown = [i[3] for i in all_metric]

  return {'neutral':neutral, 'positive':positive, 'negative':negative,'speech':speech,'skip':skip, 'unknown':unknown}, list(map(average,[positive, negative,neutral, unknown, speech,skip]))

In [30]:
columns = {'PQ106-Other':'Есть ли у Вас еще какие-либо обязанности в школе, прямо не относящиеся к Вашей психологической деятельности? Count:3694',
'PQ303-Other':'Какую психологическую документацию Вы ведете в учреждении? Count: 4202',
'PQ614':'Укажите Ваши пожелания по развитию психологической службы в системе образования Вашего субъекта? Count: 15700' }
for clmn in columns:
  result, average = analyze_sentiment(data_rus_text[clmn])
  bar_tone_analysise(['positive', 'negative','neutral', 'unknown', 'speech','skip'],average,clmn,columns[clmn] )


In [0]:
import plotly.graph_objects as go
def bar_tone_analysise_2(x,y,key='',question=''):
  # Use the hovertext kw argument for hover text
  fig = go.Figure(data=[go.Bar(x=x, y=y)])
  # Customize aspect
  fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, opacity=0.9)
  fig.update_layout(title='' + ' ' + key, width=800, height=800)
  fig.show()


In [38]:
from collections import Counter
def count_words(column):
  PQ303 = data_rus_text[column].dropna().apply(lambda x: preprocess_text(x))
  a = listmerge2(PQ303.to_list())
  a = [i.strip() for i in a if i != '']
  res = Counter(a)
  res = list(res.most_common())
  return res

for clmn in columns:
  res = count_words(clmn)
  bar_tone_analysise_2([i[0] for i in res if i[1] not in [1,2,3,4,5,6,7,8,9,10,11,12] and i[0] !=''], [i[1] for i in res if i[1] not  in [1,2,3,4,5,6,7,8,9,10,11] and i[0] !=''], key=columns[clmn])

In [0]:
print([i[0] for i in res])